# Markov Chain Monte Carlo sampling in a perfect model setting

Here we demonstrate the statistical sampling of KPP parameters using MCMC in a perfect model setting.

In [1]:
using Pkg; Pkg.activate("..")

using PyPlot, Printf, Statistics, OceanTurb, Dao, JLD2,
        ColumnModelOptimizationProject, ColumnModelOptimizationProject.KPPOptimization

In [2]:
     model_N = 30                # Model resolution
    model_dt = 10*minute         # Model timestep
initial_data = 1                 # Choose initial condition for forward runs
 target_data = (4, 7, 10)        # Target samples of saved data for model-data comparison
        case = "unstable_strong" # Case to run comparison

# Initialize the 'data' and the 'model'
 datadir = joinpath("..", "data", "coarse_perfect_model_experiment")
filepath = joinpath(datadir, case * ".jld2")

 data = ColumnData(filepath; initial=initial_data, targets=target_data)
model = KPPOptimization.ColumnModel(data, model_dt, N=model_N)

# Pick a set of parameters to optimize
defaults = DefaultFreeParameters(BasicParameters)
std = DefaultStdFreeParameters(0.05, typeof(defaults))
perturbed = NormalPerturbation(std)(defaults)

println("We will optimize the following parameters:")
@show propertynames(defaults)
@show defaults perturbed;

We will optimize the following parameters:
propertynames(defaults) = (:CRi, :CKE, :CNL, :Cτ, :Cstab, :Cunst, :Cb_U, :Cb_T)
defaults = [0.3, 4.32, 6.33, 0.4, 2.0, 6.4, 0.599, 1.36]
perturbed = [0.294718, 4.61771, 6.62473, 0.396909, 2.02005, 6.48785, 0.580094, 1.35103]


# Loss function and Log Likelihoods

A basic concept we need for MCMC sampling is the notion of a 'loss function', which in a sense represents an estimate of model error, or the discrepency between model output and data.

Our sampling strategy uses a 'composite' loss function constructed from error estimates for each field $U$, $V$, $T$, and $S$.
For a field $\Phi$ and case $i$, the loss function for a given parameter vector $\boldsymbol{C}$ is defined

$$ \mathcal{L}^i_\Phi(\boldsymbol{C}, \Phi_\mathrm{data}^i) = \int_{t_0}^{t_1}  \mathrm{d} t \sqrt{\int_{-L_z}^0 \mathrm{d} z \left ( \Phi^i_\mathrm{model} \left ( \boldsymbol{C} \right ) - \Phi^i_\mathrm{data} \right )^2 }$$

The total loss function for a case $i$ is then constructed from

$$ \mathcal{L}^i(\boldsymbol{C}, \mathrm{data}^i) = \sum_{\Phi} \mathcal{W}_\Phi \mathcal{L}^i_\Phi(\boldsymbol{C}, \Phi^i_\mathrm{data}) \, , $$

With multiple cases or sources or data (representing different physical scenarios, flux conditions, initial conditions, *et cetera*), the value of the composite loss function is, finally

$$ \mathcal{L}(\boldsymbol{C}, \Sigma \, \mathrm{data}) = \sum_i \mathcal{L}^i(\boldsymbol{C}, \mathrm{data}^i) $$

where $\mathcal{W}_\Phi$ essentially non-dimensionalizes the error associated with the comparison of different fields such as velocity $U$ or temperature $T$.

Below, we deduce the appropriate $\mathcal{W}_\Phi$ for our problem based on the velocity- and temperature-specific error assocaited with our initial (perfect) parameter choices.

Note that the object `MarkovLink` stores `(parameter, error)` pairs for a given parameter vector and `NegativeLogLikelihood` function. 

# Bayesian updates and log likelihoods

The outcome of MCMC sampling is a posterior probability density $\rho_\mathrm{post} \left (\boldsymbol{C} \, | \, \mathrm{data} \right )$, which we interpret to represent the 'likelihood' of a particular vector of parameters $\boldsymbol{C}$ being 'correct', given the data.
$\rho_\mathrm{post} \left (\boldsymbol{C} \, | \, \mathrm{data} \right )$ is defined as

$$ \rho_\mathrm{post} \left ( \boldsymbol{C} \, | \, \mathrm{data} \right ) = \exp \left [ - \mathcal{L} \left ( \boldsymbol{C} \right ) / \mathcal{L}_0 \right ] \, \, \rho_\mathrm{prior} \left (  \boldsymbol{C} \right ) \, ,$$

in terms of the loss function $\mathcal{L}(\boldsymbol{C})$, a loss function scale $\mathcal{L}_0$ (essentially reflecting the possible dimensionality of $\mathcal{L}$), and a the prior distribution $\rho_\mathrm{prior}(\boldsymbol{C})$, which may reflect some *a priori* information we may have about the parameters.
For example, given that KPP has been used for 20 years, we could assume *a priori* that the parameters are Gaussian distributed around the published parameters values.
(Here we ignore the prior, essentially assuming that $\rho_\mathrm{prior} = \mathrm{constant}$, or an 'uninformative prior'.)
Thus the negative logarithm of the likelihood of $\boldsymbol{C}$ given the data --- the 'negative log likelihood' --- is

$$ 
\begin{align}
\mathrm{NLL} &\equiv -\log \left [ \rho_\mathrm{post} \left ( \boldsymbol{C} \, | \, \mathrm{data} \right ) \right ] \\
&= \mathcal{L} \left ( \boldsymbol{C} \right ) \, / \, \underbrace{\mathcal{L}_0}_{\equiv \mathrm{"scale"}} - \underbrace{\log \left [ \rho \left ( \boldsymbol{C} \right ) \right ]}_{\equiv \mathrm{"prior"}}
\end{align}
$$

To construct a `NegativeLogLikelihood` object, we need a `model`, `data`, and a loss function.
To understand the appropriate weighting for the loss functions associated with each field $\Phi$, we first calculate loss functions based on temperature and velocity alone.

In [3]:
# Obtain an estimate of the relative error in the temperature and velocity fields
test_nll_temperature = NegativeLogLikelihood(model, data, temperature_loss)
   test_nll_velocity = NegativeLogLikelihood(model, data, velocity_loss);

The object `MarkovLink`, called with the signature

```julia
link = MarkovLink(nll::NegativeLogLikelihood, C::FreeParameters)
```
evaluates the loss function and calculated the negative log likelihood for a set of parameters.

Next, we evaluate the loss function for our chosen default parameters.

In [4]:
# Calculate error
test_link_temperature = MarkovLink(test_nll_temperature, perturbed)
   test_link_velocity = MarkovLink(test_nll_velocity, perturbed)

@show test_link_velocity.error test_link_temperature.error
@show error_ratio = test_link_velocity.error / test_link_temperature.error

test_link_velocity.error = 0.019060762141441527
test_link_temperature.error = 0.016350740652929865
error_ratio = test_link_velocity.error / test_link_temperature.error = 1.165743041617265


1.165743041617265

Finally, we build the loss function.

In [5]:
# Build the weighted NLL, normalizing temperature error relative to velocity error.
@show weights = (1, 1, 10, 0)

nll = NegativeLogLikelihood(model, data, weighted_fields_loss, weights=weights)

# Obtain the first link in the Markov chain
first_link = MarkovLink(nll, defaults)
@show nll.scale = first_link.error * 1.0

weights = (1, 1, 10, 0) = (1, 1, 10, 0)
nll.scale = first_link.error * 1.0 = 0.17369957469231553


0.17369957469231553

# Specification of Markov Chain Monte Carlo parameters

Next we demonstrate how to set up a Markov Chain Monte Carlo (MCMC) sampler.
Currently, we are only able to use the 'Hastings-Metropolis' algorithm -- a simple 
algorithm based on normally-distributed random perturbations.

The algorithm consists of three steps:

1. Generate a set of proposal parameters by normally perturbing the current parameters.
2. Evaluate the Negative Log Likelihood for the proposed parameters.
3. Decide whether to 'accept' the proposed parameters as the new current parameters on the criterion:

```julia
accept(proposal, current, scale) = current.error - proposal.error > scale * log(rand(Uniform(0, 1)))
```

Note that the right side is always negative.
Thus if the proposal error is smaller than the current error, the proposal is always accepted.
If the proposal error is larger than the current error, the proposal is *sometimes* accepted:
in particular, when the uniformly distributed random number is close to 0.
The larger tha value of 'scale', more proposals are accepted.
The code for this algorithm is:

```julia
accepted = 0
for i = 1:nlinks
    proposal = MarkovLink(nll, sampler.perturb(current.param))
    current = ifelse(accept(proposal, current, nll.scale), proposal, current)
    push!(links, proposal)
    if current === proposal
        accepted += 1
        push!(path, i)
    else
        @inbounds push!(path, path[end])
    end
end
```

That's it.

The two parameters in the MCMC algorithm are thus the 'scale' of the specified Negative Log Likelihood function, and the standard deviation of the random perturbations that generate proposal parameter sets.

Below we fix the standard deviation of the random perturbations at 5\% of the initial parameter values.

In [6]:
# Use a normal perturbation with standard deviation set to 5% of default values
std = DefaultStdFreeParameters(0.05, typeof(defaults))
bounds = BasicParameters(((0.0*p, 3.0*p) for p in defaults)...)
sampler = MetropolisSampler(BoundedNormalPerturbation(std, bounds))

ninit = 10^2
chain = MarkovChain(ninit, first_link, nll, sampler)
@show chain.acceptance chain[1].param chain[end].param

chain.acceptance = 0.76
(chain[1]).param = [0.3, 4.32, 6.33, 0.4, 2.0, 6.4, 0.599, 1.36]
(chain[end]).param = [0.068911, 8.28092, 11.3728, 0.351559, 3.87855, 6.3413, 0.369909, 2.24388]


8-element BasicParameters{Float64}:
  0.06891101425054781
  8.280920097109412  
 11.372770692978943  
  0.3515590855438135 
  3.8785506687101323 
  6.341298546358863  
  0.36990940396121613
  2.2438757682385    

# Collecting samples

Finally, we collect samples and save the data. 
Fortunately, `JLD2` is able to save the entire Markov chain as a single object with no issues.

In [7]:
dsave = 10^3
chainname = "test_markov_chain"
chainpath = "$chainname.jld2"
@save chainpath chain

tstart = time()
for i = 1:4
    tint = @elapsed extend!(chain, dsave)

    @printf("tᵢ: %.2f seconds. Elapsed wall time: %.4f minutes.\n\n", tint, (time() - tstart)/60)
    @printf("First, optimal, and last links:\n")
    println((chain[1].error, chain[1].param))
    println((optimal(chain).error, optimal(chain).param))
    println((chain[end].error, chain[end].param))
    println(" ")

    println(status(chain))

    oldchainpath = chainname * "_old.jld2"
    mv(chainpath, oldchainpath, force=true)
    @save chainpath chain
    rm(oldchainpath)
end

tᵢ: 5.86 seconds. Elapsed wall time: 0.1013 minutes.

First, optimal, and last links:
(0.17369957469231553, [0.3, 4.32, 6.33, 0.4, 2.0, 6.4, 0.599, 1.36])
(0.10735228550263193, [0.289194, 2.72196, 7.04917, 0.498073, 5.6378, 15.3149, 1.05791, 0.994302])
(0.2768113776928492, [0.0336146, 9.81828, 18.779, 0.659273, 3.48409, 13.9781, 0.426778, 2.25608])
 
               length | 1100
           acceptance | 0.715454545
 initial scaled error | 1.000000000
 optimal scaled error | 0.618034245

tᵢ: 5.92 seconds. Elapsed wall time: 0.2061 minutes.

First, optimal, and last links:
(0.17369957469231553, [0.3, 4.32, 6.33, 0.4, 2.0, 6.4, 0.599, 1.36])
(0.10735228550263193, [0.289194, 2.72196, 7.04917, 0.498073, 5.6378, 15.3149, 1.05791, 0.994302])
(1.5330236436183398, [0.560433, 4.01063, 11.8763, 0.0400355, 1.8529, 6.949, 0.231212, 1.85852])
 
               length | 2100
           acceptance | 0.616666667
 initial scaled error | 1.000000000
 optimal scaled error | 0.618034245

tᵢ: 6.22 seconds. El

Finally, we show how to create a `BatchedNegativeLogLikelihood` function.

In [8]:
cases = ("free_convection", "unstable_weak", "unstable_strong", "stable_weak", "stable_strong", "neutral")

  datadir = joinpath("..", "data", "perfect_model_experiment")
filepaths = Dict((case, joinpath(datadir, case * ".jld2")) for case in cases)
 datasets = Dict((case, ColumnData(filepaths[case]; initial=initial_data, targets=target_data)) for case in cases)
   models = Dict((case, KPPOptimization.ColumnModel(datasets[case], model_dt, N=model_N)) for case in cases)

defaults = DefaultFreeParameters(BasicParameters)
@show weights = (1, 1, 10, 0)

# Build the batch of NLLs.
nll = BatchedNegativeLogLikelihood(
    [ NegativeLogLikelihood(models[case], datasets[case], weighted_fields_loss, weights=weights)
        for case in cases ])

# Obtain the first link in the Markov chain
first_link = MarkovLink(nll, defaults)
nll.scale = first_link.error * 1.0

std = DefaultStdFreeParameters(0.05, typeof(defaults))
bounds = BasicParameters(((0.0*p, 3.0*p) for p in defaults)...)
sampler = MetropolisSampler(BoundedNormalPerturbation(std, bounds))

chainname = "test_batch_markov_chain"
chainpath = "$chainname.jld2"
dsave = 10^2
chain = MarkovChain(dsave, first_link, nll, sampler)
@save chainpath chain

tstart = time()
for i = 1:4
    tint = @elapsed extend!(chain, dsave)

    @printf("tᵢ: %.2f seconds. Elapsed wall time: %.4f minutes.\n\n", tint, (time() - tstart)/60)
    @printf("First, optimal, and last links:\n")
    println((chain[1].error, chain[1].param))
    println((optimal(chain).error, optimal(chain).param))
    println((chain[end].error, chain[end].param))
    println(" ")

    println(status(chain))

    oldchainpath = chainname * "_old.jld2"
    mv(chainpath, oldchainpath, force=true)
    @save chainpath chain
    rm(oldchainpath)
end

weights = (1, 1, 10 * round(Int, mean(values(ratios)) / 10), 0) = (1, 1, 20, 0)
first_link.error = 0.2611217428337403
chain.acceptance = 0.67


0.67